#1. Грузим данные в нужном формате, описываем структуру сети и обучаем ее.

In [1]:
from torchvision import transforms
from torchvision import datasets
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms
from torchvision import datasets


In [2]:
# random seed and output dir setup
import random

frame_size = (224, 224)

batch_size = 64
random_seed = 42
classes_header = ["0", "1", "2", "3", "4"] 
test_dataset_path = 'oai224/test'
train_dataset_path = 'oai224/train'
checkpoints_dir = 'DenseNet121_42/'
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# if you are suing GPU
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
from sklearn import model_selection

validation_split = .125

# Разделяем данные на тренировочные и валидационные
transforms_to_train = transforms.Compose([         
              transforms.ColorJitter(brightness=.33, saturation=.33),
              transforms.RandomHorizontalFlip(p=0.5),
              transforms.RandomAffine(degrees=(-10, 10), scale=(0.9, 1.10)),
              transforms.Resize(frame_size), 

              transforms.ToTensor(),
              transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ])

train_dataset = datasets.ImageFolder(train_dataset_path, transform=transforms_to_train)
targets = train_dataset.targets

train_idx, valid_idx = model_selection.train_test_split(
    np.arange(len(train_dataset.targets)), test_size=validation_split, random_state=42, shuffle=True, stratify=targets)

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, drop_last=True)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler, drop_last=True)


In [11]:

if torch.cuda.is_available(): # Let's make sure GPU is available!
    device = torch.device("cuda:0")
    device_name = 'cuda:0'
    print("device: CUDA")
else:
    device = torch.device("cpu")
    device_name = 'cpu'
    print("device: CPU")


device: CPU


In [12]:
def weighted_loss(outputs, labels):
    softmax_op = torch.nn.Softmax(1)
    prob_pred = softmax_op(outputs)

    def set_weights():
        # weight matrix 04 (wm04)
        init_weights = np.array([[1, 3, 6, 7, 9],
                                 [4, 1, 4, 5, 7],
                                 [6, 4, 1, 3, 5],
                                 [7, 5, 3, 1, 3],
                                 [9, 7, 5, 3, 1]], dtype=np.float)

        adjusted_weights = init_weights + 1.0
        np.fill_diagonal(adjusted_weights, 0)

        return adjusted_weights
    
    cls_weights = set_weights()

    batch_num, class_num = outputs.size()
    class_hot = np.zeros([batch_num, class_num], dtype=np.float32)
    labels_np = labels.data.cpu().numpy()
    
    for ind in range(batch_num):
        class_hot[ind, :] = cls_weights[labels_np[ind], :]
    class_hot = torch.from_numpy(class_hot)
    class_hot = torch.autograd.Variable(class_hot).cuda()

    loss = torch.sum((prob_pred * class_hot)**2) / batch_num
    # loss = torch.mean(prob_pred * class_hot)

    return loss

In [13]:
import re
import os
import utils

def append_to_file(filename, val):
    with open(filename, 'a') as f:
        f.write("%s\n" % val)

def read_file(filename):
    lines = []
    with open(filename, 'r') as f:
        lines = [float(line.strip()) for line in f]

    return lines

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, lr_scheduler = None, anneal_epoch = 0): 
    if not os.path.exists(checkpoints_dir):
        os.makedirs(checkpoints_dir)

    start_epoch = 0

    open(checkpoints_dir + 'loss_history.txt', 'w').close()
    open(checkpoints_dir + 'train_history.txt', 'w').close()
    open(checkpoints_dir + 'val_history.txt', 'w').close()
    open(checkpoints_dir + 'best_accuracy.txt', 'w').close()
    print("start traning from scratch...")

    best_val_accuracy = 0

    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0

        # process batches
        for i_step, (x, y) in enumerate(train_loader): 
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    

            loss_value = loss(prediction, y_gpu)
            # loss_value = weighted_loss(prediction, y_gpu)

            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        # check accuracy
        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples

        val_accuracy = 0.0
        with torch.no_grad():
          val_accuracy, _, _, _ = utils.compute_accuracy(model, val_loader)

        # write marks to files
        append_to_file(checkpoints_dir + 'loss_history.txt', float(ave_loss))
        append_to_file(checkpoints_dir + 'train_history.txt', train_accuracy)
        append_to_file(checkpoints_dir + 'val_history.txt', val_accuracy)

        # update learning rate
        if lr_scheduler is not None and epoch >= anneal_epoch:
          lr_scheduler.step()
        
        stage = epoch + start_epoch

        if val_accuracy > best_val_accuracy:
          best_val_accuracy = val_accuracy
          
          # next better model
          model_save_name2 = 'model.ckpt-' + str(stage)
          torch.save(model.state_dict(), checkpoints_dir + F"{model_save_name2}")
          
          # best model
          model_save_name = 'best_model.ckpt'
          torch.save(model.state_dict(), checkpoints_dir + F"{model_save_name}")

          append_to_file(checkpoints_dir + 'best_accuracy.txt', best_val_accuracy)
          print("update best model with val. accuracy %f on stage %d" % (best_val_accuracy, stage))

        print("epoch %d; average loss: %f, train accuracy: %f, val accuracy: %f" % (stage, ave_loss, train_accuracy, val_accuracy))
        
    print("final best accuracy: %f" % (best_val_accuracy))

    return loss_history, train_history, val_history
        

In [ ]:
import custom_models

cnn_model = custom_models.densenet121_model(5, use_pretrained=True)
# cnn_model = custom_models.densenet161_model(num_class=5, use_pretrained=True)
# cnn_model = custom_models.AntonyCnn(num_classes=2)

# train model
if device_name.startswith('cpu'):
    cnn_model.type(torch.FloatTensor)
    cnn_model.to(device)
else:
    cnn_model.type(torch.cuda.FloatTensor)
    cnn_model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(cnn_model.parameters(), lr=1e-3, weight_decay=1e-4) # default: weight_decay=1e-4
# optimizer = optim.SGD(cnn_model.parameters(), lr=1e-4, momentum=0.9, nesterov=True) 

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95) # decrease lr by 10% every 7 epochs

loss_history, train_history, val_history = train_model(cnn_model, train_loader, val_loader, loss, optimizer, 71, lr_scheduler)


start traning from scratch...
update best model with val. accuracy 0.385417 on stage 0
epoch 0; average loss: 1.515926, train accuracy: 0.367188, val accuracy: 0.385417


In [ ]:
import matplotlib.pyplot as plt

train_history = read_file(checkpoints_dir + 'train_history.txt')
val_history = read_file(checkpoints_dir + 'val_history.txt')

# vizualize accuracy
plt.plot(train_history)
plt.plot(val_history)
plt.savefig(checkpoints_dir + 'graph.png')